In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import gan_protein_structural_requirements.utils as utils
from gan_protein_structural_requirements.utils.protein_visualizer import jupy_viz_obj as jvo, jupy_viz_file as jvf
from gan_protein_structural_requirements.utils.folding_models import esm_batch_predict as esm_batch_predict, esm_predict as esm_predict, load_tokenizer as load_tokenizer
from gan_protein_structural_requirements.utils.extract_structure import extract_structures as extract_structures, untokenize as untokenize, get_untokenizer as get_untokenizer
from gan_protein_structural_requirements.data import class_protein_dataset as cpd

In [ ]:
dtst = cpd.Protein_dataset(
    root_dir="./gan_protein_structural_requirements/data/raw/batch_1_data/",
    path_to_esm="/Users/rohitkulkarni/Documents/gan_protein_structural_requirements/gan_protein_structural_requirements/utils/esmfoldv1"
    )

In [ ]:
dtst.Y